In [1]:
data_var = '2024-01-30'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10933,2024-01-30,Eua Nba,00:00,Portland Trail Blazers,Philadelphia 76ers,3.93,1.28,220.5,1.85,2.09,8.5,1.97,1.97,f1tlVskq,0.254453,0.781250,0.540541,0.478469,0.035703,0.4,0.6,NaN,NaN,566.750,280.261665,0.494507,185.016,64.293126,0.347500,357.046,218.060,425.98,287.70,1.0,0.0,0.0,0.0,0.719322,0.086145,0.000000,3.19,0.638,4.592476,0.397397,0.5,0.102603,-2.24,-0.448,-0.625000,0.590252,0.5,-0.090252
10934,2024-01-30,Europa Euroliga,16:00,Bayern,Baskonia,1.79,2.07,162.5,1.82,1.92,-3.5,2.03,1.73,WC8jfMr8,0.558659,0.483092,0.549451,0.520833,0.041751,0.6,0.6,NaN,NaN,192.208,115.867209,0.602822,211.080,89.473058,0.423882,146.702,245.566,82.56,240.24,0.0,0.0,0.0,0.0,0.102585,0.037813,0.112836,-1.93,-0.386,-2.046632,0.000000,0.0,0.000000,-0.18,-0.036,-29.722222,0.000000,0.0,0.000000
10935,2024-01-30,Europa Euroliga,16:00,Crvena zvezda,Valencia,1.40,3.08,158.5,1.88,1.88,-7.5,2.03,1.75,vJL4WIj1,0.714286,0.324675,0.531915,0.531915,0.038961,0.4,0.4,NaN,NaN,149.158,21.915472,0.146928,203.448,130.192598,0.639931,147.944,264.808,135.90,203.28,0.0,0.0,0.0,0.0,0.530330,0.000000,0.104757,-0.36,-0.072,-5.555556,0.000000,0.0,0.000000,4.60,0.920,2.260870,0.000000,0.0,0.000000
10936,2024-01-30,Europa Euroliga,16:00,Lyon-Villeurbanne,Fenerbahce,3.03,1.41,164.5,1.86,1.96,4.5,2.07,1.70,b747Xbye,0.330033,0.709220,0.537634,0.510204,0.039253,0.8,0.8,NaN,NaN,199.240,73.436347,0.368582,133.674,17.653881,0.132067,188.042,170.638,91.35,125.25,0.0,0.0,0.0,0.0,0.515997,0.037021,0.138795,-0.97,-0.194,-10.463918,0.495673,0.2,-0.295673,-3.59,-0.718,-0.571031,0.000000,0.0,0.000000
10937,2024-01-30,Europa Euroliga,16:15,Olympiakos,Alba Berlin,1.16,5.60,157.5,1.85,1.93,-12.5,2.01,1.78,I9K8Vx67,0.862069,0.178571,0.540541,0.518135,0.040640,0.2,0.4,NaN,NaN,236.994,202.098964,0.852760,239.676,84.312242,0.351776,106.128,350.268,134.39,253.44,0.0,0.0,0.0,0.0,0.928862,0.029930,0.085823,0.80,0.160,1.000000,0.702248,0.7,-0.002248,-2.12,-0.424,-10.849057,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11021,2024-01-30,Eua Ncaa,21:00,St. Bonaventure,VCU Rams,1.58,2.47,137.5,1.91,1.91,-5.5,2.05,1.71,drCLqqn3,0.632911,0.404858,0.523560,0.523560,0.037770,0.8,0.0,NaN,NaN,119.218,21.639823,0.181515,114.838,10.602279,0.092324,112.310,0.000,149.24,113.40,0.0,0.0,0.0,0.0,0.310778,0.000000,0.127881,0.66,0.132,4.393939,0.725145,0.7,-0.025145,0.00,0.000,inf,0.000000,0.0,0.000000
11022,2024-01-30,Eua Ncaa,21:00,Villanova,Marquette,1.70,2.24,143.5,1.91,1.91,-2.5,2.00,1.71,fak8JFv0,0.588235,0.446429,0.523560,0.523560,0.034664,0.2,0.6,NaN,NaN,144.024,39.514021,0.274357,103.004,26.122985,0.253611,99.838,104.592,144.95,88.58,0.0,0.0,0.0,0.0,0.193826,0.000000,0.110545,-0.97,-0.194,-3.608247,0.753689,0.7,-0.053689,-2.17,-0.434,-2.857143,0.000000,0.0,0.000000
11023,2024-01-30,Eua Ncaa,22:00,Loyola (Ill).,St. Louis,1.21,4.67,146.5,1.80,1.95,-10.5,2.00,1.71,ru8HpPWd,0.826446,0.214133,0.555556,0.512821,0.040579,0.2,0.4,NaN,NaN,152.560,55.744602,0.365395,291.466,224.069525,0.768767,103.658,448.232,136.67,226.92,0.0,0.0,0.0,0.0,0.832173,0.056569,0.110545,0.67,0.134,1.567164,0.694259,0.8,0.105741,-5.00,-1.000,-3.670000,0.000000,0.0,0.000000
11024,2024-01-30,Eua Ncaa,23:00,Depaul,Seton Hall,4.91,1.19,139.5,1.91,1.91,8.5,2.10,1.67,CzpDIZg6,0.203666,0.84

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,Philadelphia 76ers,220.5,1.85,1.0000,Over
1,20:30,Eua Ncaa,Tennessee,South Carolina,137.5,2.00,0.8571,Over
2,21:30,Eua Nba,Atlanta Hawks,Los Angeles Lakers,245.5,1.80,1.0000,Over
3,21:30,Eua Nba,Boston Celtics,Indiana Pacers,242.5,1.88,1.0000,Over
4,21:30,Eua Nba,New York Knicks,Utah Jazz,230.5,1.88,1.0000,Over
